In [3]:
# create environment variables
import os
os.environ['MAPS_API_KEY'] = ""
os.environ["OPENAI_API_KEY"] = ""

In [4]:
import requests  
from openai import OpenAI
from functions.settings import get_settings
import json
  
def get_directions(api_key, start, end):  
    # start and end are place ids
    directions_url = "https://maps.googleapis.com/maps/api/directions/json"  
    params = {
        'origin': f'place_id:{start}',
        'destination': f'place_id:{end}',
        'key': api_key
    }
      
    response = requests.get(directions_url, params=params)  
    directions = response.json()  
    return directions  
  
def get_notable_sights(api_key, path):  
    places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
    notable_sights = []  
      
    # Extract points from the path  
    points = [step['end_location'] for step in path['legs'][0]['steps']]  
    
    # specify such included types that will make sense to be used to generate a 
    # riddle. for example, the "route" type is not included because it is not
    # a place thatthat make for a good riddle fodder.
    included_types = [
        "amusement_park",
        "aquarium",
        "art_gallery",
        "bakery",
        "bar",
        "book_store",
        "cafe",
        "campground",
        "church",
        "hindu_temple",
        "mosque",
        "city_hall",
        "clothing_store",
        "convenience_store",
        "department_store",
        "electronics_store",
        "florist",
        "furniture_store",
        "grocery_or_supermarket",
        "hardware_store",
        "home_goods_store",
        "jewelry_store",
        "liquor_store",
        "meal_delivery",
        "meal_takeaway",
        "movie_theater",
        "museum",
        "night_club",
        "park",
        "restaurant",
        "shopping_mall",
        "spa",
        "store",
        "zoo",
        "route",
        "political",
        "point_of_interest",
    ]
      
    for point in points:  
        params = {  
            'location': f"{point['lat']},{point['lng']}",  
            'radius': 10,  
            'key': api_key,  
            # 'type': "|".join(included_types)
        }
          
        response = requests.get(places_url, params=params)  
        results = response.json().get('results', [])  
          
        for place in results:  
            notable_sights.append({  
                'name': place.get('name'),  
                'location': place.get('geometry', {}).get('location')  
            })
      
    return notable_sights  
  
def generate_riddle(api_key, start, end):  
    # Get directions and notable sights
    directions = get_directions(api_key, start, end)  
    notable_sights = get_notable_sights(api_key, directions['routes'][0])  
    # Generate a riddle based on the notable sights  
    # get the place details for the start and end points
    places_url = "https://maps.googleapis.com/maps/api/place/details/json"

    # use only the name key in the notable_sights list
    notable_sights = [sight['name'] for sight in notable_sights]

    params = {
        'place_id': start,
        'key': api_key
    }

    response = requests.get(places_url, params=params)
    start_name = response.json().get('result', {}).get('name')

    params = {
        'place_id': end,
        'key': api_key
    }

    response = requests.get(places_url, params=params)
    end_name = response.json().get('result', {}).get('name')

    prompt = (
        "You have a list of some notable points that lie in a path leading to some destination. "
        "Build a riddle that uses some of these points (the ones you think are best suited to be used) as "
        "clues to the destination. The riddle should be such that the answer to the riddle is the destination. "
        "Don't mention the destination name in the riddle. "
        f"The start location is {start_name} and more importantly, the destination is {end_name}. "
        f"The notable points are as follows: {notable_sights}."
        "Only output the riddle and nothing else."
    )
    
    openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
    )

    riddle = response.choices[0].message.content
    # TODO generate image and voice over to the riddle
    return riddle


ImportError: cannot import name 'get_settings' from 'functions.settings' (/home/wjayesh/apps/places/functions/settings.py)

In [5]:
INSTRUCTIONS = [
    "You are a game master and must help conduct a game for the user."
    "Whenever, the start_game function is called, you must always have user's "
    "interests with you. If the message doesn't have user's interests, you MUST "
    "ask the user for their interests. Once the interests are clear, you can call "
    "the start_game function with the user's interests and the current location. "
    "There will be other functions that you may call according to the tool descriptions."
]

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "start_game",
            "description": (
                "This should be called when a user wants to start a game. "
                "The input parameters should be the user's current location "
                "and the user's interests."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "current_location": {
                        "type": "string",
                        "description": "The current location that is passed as part of the input message.",
                    },
                    "user_interests": {
                        "type": "string",
                        "description": "The users' interests that are passed as part of the input message.",
                    }
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "verify",
            "description": "Retrieves the details of a specific post from the forum. The tool should take the post ID as input and return the details of the post including the content, author, date, and other relevant information. It should be used when the user asks to read a specific post.",
            "parameters": {
                "type": "object",
                "properties": {
                    "current_location": {
                        "type": "string",
                        "description": "The current location of the user.",
                    },
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_hint",
            "description": "Generates a hint for the user to solve the current riddle and target location.",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
]

In [6]:
from julep import Client
api_key = ""
    
client = Client(api_key=api_key)

In [7]:
agent = client.agents.create(
        name="RiddleMaster",
        about="An agent that will act as a game master who can conduct a riddle based game where you find the next destination to go to",
        tools=TOOLS,
        instructions=INSTRUCTIONS,
        model="gpt-4o",
        default_settings={
            "temperature": 0.5,
            "top_p": 1,
            "min_p": 0.01,
            "presence_penalty": 0,
            "frequency_penalty": 0,
            "length_penalty": 1.0,
        },
        metadata={"name": "RiddleMaster"},
)

SITUATION_PROMPT = """
You are RiddleMaster, an Agent that can conduct games for users. Users will send you messages
with different intents. They can be about starting a new game, verifying if the locations they have
guessed are correct, or if they want a hint. You will use the tools at your disposal to help the users
and make the game experience enjoyable.
Follow the instructions strictly.
"""

session = client.sessions.create(
    agent_id=agent.id,
    situation=SITUATION_PROMPT,
    metadata={"agent_id": agent.id},
)


In [11]:

response = client.sessions.chat(
    session_id=session.id,
    messages=[
        {
            "role": "user",
            "content": "my interests are in cafes and parks.",
        }
    ],
    recall=True,
    remember=True,
)

In [12]:
response

ChatResponse(id='b041f743-6b4e-4c57-a491-a00a090422c2', finish_reason=<ChatResponseFinishReason.TOOL_CALLS: 'tool_calls'>, response=[[ChatMlMessage(role=<ChatMlMessageRole.FUNCTION_CALL: 'function_call'>, content='{"arguments":"{\\"current_location\\":\\"12.76,45.89\\",\\"user_interests\\":\\"cafes and parks\\"}","name":"start_game"}', name='RiddleMaster', created_at=datetime.datetime(2024, 6, 3, 13, 19, 26, 724673, tzinfo=datetime.timezone.utc), id='fcde103d-87ad-4eb2-9bb6-a3385c51932e')]], usage=CompletionUsage(completion_tokens=30, prompt_tokens=454, total_tokens=484), jobs=None, doc_ids=DocIds(agent_doc_ids=[], user_doc_ids=[]))

In [13]:
import json
from functions.start import start_game, get_hint, verify

if response.finish_reason == "tool_calls":
    json_response = json.loads(response.response[0][0].content)
    function_name = json_response["name"]
    function_params = json.loads(json_response["arguments"])

    if function_name == "start_game":
        current_location = function_params["current_location"]
        user_interests = function_params["user_interests"]
        riddle = start_game(current_location, user_interests)
        print(riddle)

    if function_name == "verify":
        current_location = function_params["current_location"]
        response = verify(current_location)
        print(response)

    if function_name == "get_hint":
        hint = get_hint()
        print(hint)
        

All places {}


IndexError: list index out of range

In [27]:
prompt = (
        "This list of interests come from a group of people interested in "
        "exploring a city. Take this list of interests and generate exact keywords "
        "to pass to the Google Maps Places API to get the coordinates for them. The "
        "interests are: {}. The keywords should be a list of strings. "
        "Output only the list and nothing else ".format("i like to work out of cafes and parks")
    )

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]
)

In [32]:
response.choices[0].message.content.split(", ")

['cafe', 'park']

In [20]:
response.response[0][0].content

'{"arguments":"{\\"current_location\\":\\"32.715736,-117.161087\\",\\"user_interests\\":\\"cafes that I can work out of, and parks\\"}","name":"start_game"}'

In [50]:
os.environ["OPENAI_API_KEY"] = ""

In [8]:
# reload file
import importlib
from functions import generate_riddle_function, start

importlib.reload(start)
importlib.reload(generate_riddle_function)

<module 'functions.generate_riddle_function' from '/home/wjayesh/apps/places/functions/generate_riddle_function.py'>